In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

In [14]:
import joblib
from pathlib import Path
import os
import cv2

from utils.render import render_w_pytorch3d
from utils.smpl_deformer.smpl_server import SMPLServer
from utils.io import load_frame_map_jsonl_restore
from preprocess.helpers.cameras import load_default_camdicts
from preprocess.helpers.video_utils import load_images

import numpy as np
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt

In [4]:
output_dir = "/scratch/izar/cizinsky/thesis/output/modric_vs_ribberi"
frame_path_path = f"{output_dir}/preprocess/frame_map.jsonl"
scene_root = f"{output_dir}/preprocess/"
h4d_results = load_frame_map_jsonl_restore(frame_path_path, scene_root=scene_root)

phalp_res_path = f"{output_dir}/phalp_v2/results/demo_images.pkl"
default_cam_dicts = load_default_camdicts(phalp_res_path)


img_path = os.path.join(scene_root, "images")
rgb_images = load_images(img_path)

In [5]:
smpl_server = SMPLServer()
out = render_w_pytorch3d(
    default_cam_dicts,
    h4d_results,
    smpl_server,
    smpl_server.smpl.faces,
    zoom_scale=1
)

Render (PyTorch3D):  97%|█████████▋| 84/87 [00:03<00:00, 25.46it/s]

Bin size was too small in the coarse rasterization phase. This caused an overflow, meaning output may be incomplete. To solve, try increasing max_faces_per_bin / max_points_per_bin, decreasing bin_size, or setting bin_size to 0 to use the naive rasterization.Bin size was too small in the coarse rasterization phase. This caused an overflow, meaning output may be incomplete. To solve, try increasing max_faces_per_bin / max_points_per_bin, decreasing bin_size, or setting bin_size to 0 to use the naive rasterization.Bin size was too small in the coarse rasterization phase. This caused an overflow, meaning output may be incomplete. To solve, try increasing max_faces_per_bin / max_points_per_bin, decreasing bin_size, or setting bin_size to 0 to use the naive rasterization.Bin size was too small in the coarse rasterization phase. This caused an overflow, meaning output may be incomplete. To solve, try increasing max_faces_per_bin / max_points_per_bin, decreasing bin_size, or setting bin_size 

Render (PyTorch3D): 100%|██████████| 87/87 [00:03<00:00, 25.96it/s]


In [21]:
fids = sorted(out.keys())
@interact(i=IntSlider(min=0, max=len(fids)-1, step=1, value=0))
def show(i=0):
    fid = fids[i]

    rend = out[fid]  # (H,W,4), RGBA, uint8
    smpl_rgb = rend[..., :3].astype(np.float32) / 255.0
    a = rend[..., 3:4].astype(np.float32) / 255.0   # (H,W,1)

    frame = rgb_images[fid].astype(np.float32) / 255.0
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    comp = smpl_rgb * a + frame * (0.8 - a)
        
    plt.figure(figsize=(10,6))
    plt.imshow(comp)
    plt.axis('off')

interactive(children=(IntSlider(value=0, description='i', max=86), Output()), _dom_classes=('widget-interact',…